# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [13]:
import pandas as pd
import numpy as np
from scipy import stats

In [14]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [15]:
data.shape

(4870, 65)

In [16]:
# number of callbacks for each group
w_callback = sum(data[data.race=='w'].call)
b_callback = sum(data[data.race=='b'].call)
print('# of white-sounding candidates called back = ', w_callback)
print('# of black-sounding candidates called back = ', b_callback)

# of white-sounding candidates called back =  235.0
# of black-sounding candidates called back =  157.0


So it looks like, 235 candidates with white-sounding names were called back, whereas only 157 candidates with black-sounding names were called back.

In [17]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


# 1. What test is appropriate for this problem? Does CLT apply?

Since we have two sample groups in our data, we will perform a 2-sample test. The test will help us determine whether race has a significant impact on the rate of callbacks for resumes or not. 

The CLT would apply because the sample size is much larger than 30 (n=4870) and large enough that the number of successes and failures will be sufficiently large. The sample size is much less than 10% of the total job-seeking population making it an independent sample. Independence can also be assumed because the decision to callback one candidate does not affect the decision to callback another candidate. The data is assumed to be randomly sampled.

# 2. What are the null and alternate hypotheses?

The null hypothesis is that the population proportion of black-sounding names (pb) who received callbacks is equal to the population proportion of white-sounding names (pw) who received callbacks.

The alternative hypothesis is that the population proportion of black-sounding names (pb) who received callbacks is not equal to the population proportion of white-sounding names (pw) who received callbacks. 

$H_0$:pw-pb = 0

$H_1$:pw-pb $\ne$ 0

We assume that the null hypothesis is true.

In [18]:
w = data[data.race=='w']
b = data[data.race=='b']

In [19]:
print('# of white-sounding candidates = ', w.shape[0])
print('# of black-sounding candidates = ', b.shape[0])

# of white-sounding candidates =  2435
# of black-sounding candidates =  2435


Of the 4870 rows that we have in our dataset, 2435 are marked white-sounding and 2435 are marked black-sounding. So, each group has equal number of data points.

# 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [20]:
pw = w_callback/w.shape[0]
pb = b_callback/b.shape[0]
print("% of white-sounding resumes that were called back =", round(pw*100,2))
print("% of black-sounding resumes that were called back =", round(pb*100,2))

% of white-sounding resumes that were called back = 9.65
% of black-sounding resumes that were called back = 6.45


First we will try the frequestist statistical approach.

In [21]:
# Compute z-score for 99% Confidence Interval
z_score = stats.norm.ppf(.995)

# Compute Margin of Error
margin_error = z_score*np.sqrt(pw*(1-pw)/w.shape[0]+pb*(1-pb)/b.shape[0])
print("Margin of Error =", margin_error)

Margin of Error = 0.02004863403754258


In [22]:
# estimate the population proportions for the null hypothesis
p0 = np.sum(data.call) / len(data.call)

nw = len(w.call)
nb = len(b.call)

# Compute degrees of freedom
dof = nw + nb - 2
print('Degrees of freedom =', dof)

# find the difference between sample proportions
pw = np.mean(w.call)
pb = np.mean(b.call)
p_diff = pw-pb
print('Observed proportion difference =', p_diff)

# find the standard deviations
w_std = w.call.std()
b_std = b.call.std()
std = np.sqrt(((nw - 1)*w_std**2 + (nb - 1)*b_std**2) / dof)
std_diff = np.sqrt((w_std)**2 + (b_std)**2)
print('Standard deviation =', round(std, 3))

# confidence level
alpha = 0.01
print('Alpha for 99% confidence level =', alpha)

# calculate interval
ci_low = p_diff - margin_error
ci_high = p_diff + margin_error

print("99% Confidence interval =", (ci_low, ci_high))

Degrees of freedom = 4868
Observed proportion difference = 0.03203285485506058
Standard deviation = 0.272
Alpha for 99% confidence level = 0.01
99% Confidence interval = (0.011984220817517998, 0.05208148889260315)


In [23]:
z_score2 = p_diff/np.sqrt(pw*(1-pw)/nw+pb*(1-pb)/nb)
p = 1-stats.norm.cdf(z_score2)
print("The probability that the difference in proportion is due to chance is", p, ".")

The probability that the difference in proportion is due to chance is 1.9312819064531972e-05 .


So we see above that the p-value is much smaller than the significance level in the z-test.

Next we will try the bootstrapping approach.

In [24]:
# seed the pseudorandom number generator
np.random.seed(42)

# set the number of replicates to generate
# NREPS = 10000

# initialize an array for the bootstrap replicates
bs_reps = np.empty(10000)

# generate 10000 bootstrap replicates for pw - pb
bs_reps_w = np.random.binomial(nw, p0, size=10000) / nw
bs_reps_b = np.random.binomial(nb, p0, size=10000) / nb
bs_reps = bs_reps_w - bs_reps_b
    
# compute the 99% confidence interval
bs_ci = np.percentile(bs_reps + p_diff, [0.5, 99.5])
print('99% Confidence Interval is', tuple(bs_ci))

# compute the margin of error
bs_me = (bs_ci[1] - bs_ci[0]) / 2
print('Margin of Error =', bs_me)

# compute the p-value
p_bs = np.sum(bs_reps >= p_diff) / len(bs_reps)
print('p-value =', p_bs)

99% Confidence Interval is (0.011909651569639634, 0.052566735758551336)
Margin of Error = 0.02032854209445585
p-value = 0.0


# 4. Write a story describing the statistical significance in the context or the original problem.

A study conducted to examine racial discrimination in the United States labor market found that only 6.45% of resumes with black-sounding names received requests for interviews. The callback rate was 1.5 times higher (9.65%) when white-sounding names were assigned to identical resumes.

The data was explored using frequentist and bootstrapping methods.

A $p$-value was obtained for each of two hypothesis tests. The bootstrap hypothesis test gave a $p$-value of $0$, indicating that none of the 10,000 generated bootstrap replicates were at least as extreme as the difference between the observed sample proportions. The $z$-test gave a $p$-value of $1.93 \times 10^{-5}$, indicating that there is a less than 1 % chance of observing a value as extreme as the difference between the sample proportions or greater under the null hypothesis. Since both the tests resulted in p-values below the 0.01 alpha value, the null hypotheses should be rejected. 

A 99% confidence interval was evaluated for each of two different approaches. The corresponding margin of error was evaluated for each confidence interval as well. Both approaches had similar results, with a confidence interval of $(0.01, 0.05)$ and a margin of error of $0.02$. These results suggest that, with 99% confidence, the true difference between the population proportions of callbacks for white sounding names and black sounding names is within this interval. The interval does not include 0, so the confidence intervals suggest that the two population proportions are not equal. 

Therefore, we reject our null hypothesis that there is no actual difference in callback rates between black and white candidates. Those with "white-sounding" names receive more calls from employeers than those with "black-sounding" names. 
Through this analysis, we conclude that there is a statistically significant difference in the callback rates for resumes with white-sounding names when compared to black-sounding names. Racial discrimination continues to exist in the United States labor market.

# 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

The analysis shows that race could be an important factor in callback success. However that does not necessarily mean that it is the "most" important factor. There are several other variables in this dataset that were not checked for this analysis. Education level, years of experience etc on the resume also influence the callback rate. Also the data was collected for only 2 cities. 

In the future, it would be useful to gather data from more cities and it should be analyzed using other independent variables  in addition to race.